<a href="https://colab.research.google.com/github/Tonge-Shim/1/blob/master/xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pandas import DataFrame
from sklearn.metrics import classification_report
import xgboost as xgb

warnings.filterwarnings(action='ignore')
%matplotlib inline

In [2]:
df= pd.read_csv('samp_cst_feat.csv')
dtype = pd.read_excel('variable_dtype.xlsx')
dtype_categ = dtype[(dtype['dType']=='categorical')]
dtype_numrc = dtype[(dtype['dType']=='numerical')]
numrc_var = dtype_numrc['Variable_Name']
categ_var = dtype_categ['Variable_Name']
numrc_var = numrc_var.values.tolist()
categ_var = categ_var.values.tolist()#0,1
x=df.drop(categ_var, axis=1).drop(['cst_id_di'],axis=1)#10124개
y=pd.read_csv('samp_train.csv').drop(['cst_id_di'],axis=1)#10124
y_nozero= y[(y['MRC_ID_DI']!=0)]#not 0!!!1924
#y_nozeroindex=y_nozero.loc[1:10119]
y_nozeroindexlist=y_nozero.index.tolist()
x_nozero=x.loc[y_nozeroindexlist]

In [3]:
print(x_nozero.shape)
print(y_nozero.shape)
print(x.shape)
print(y.shape)
y_nozeroarray=y_nozero.to_numpy()#1924
y_array=y.to_numpy()#10124
x_nozeroarray=x_nozero.to_numpy()#1924
x_array=x.to_numpy()#10124

(1924, 198)
(1924, 1)
(10124, 198)
(10124, 1)


#0을 제외한 1924개의 xgboost

In [5]:
x_train, x_test, y_train, y_test=x_nozeroarray[:1400], x_nozeroarray[1400:], y_nozeroarray[:1400], y_nozeroarray[1400:]
shuffle_index=np.random.permutation(1400)
x_train, y_train=x_train[shuffle_index], y_train[shuffle_index]

In [6]:
#Train the XGboost Model for Classification
import xgboost as xgb
model1 = xgb.XGBClassifier()
model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)

train_model1 = model1.fit(x_train, y_train)
train_model2 = model2.fit(x_train, y_train)


In [7]:
#prediction and Classification Report
from sklearn.metrics import classification_report

pred1 = train_model1.predict(x_test)
pred2 = train_model2.predict(x_test)

print('Model 1 XGboost Report %r' % (classification_report(y_test, pred1)))
print('Model 2 XGboost Report %r' % (classification_report(y_test, pred2)))

Model 1 XGboost Report '              precision    recall  f1-score   support\n\n           1       0.25      0.08      0.12        25\n           2       0.00      0.00      0.00         2\n           3       0.00      0.00      0.00        12\n           4       0.00      0.00      0.00         5\n           5       0.29      0.11      0.16        35\n           6       0.40      0.36      0.38       156\n           7       0.45      0.76      0.56       173\n           8       0.33      0.25      0.29        88\n           9       0.00      0.00      0.00        19\n          10       0.00      0.00      0.00         9\n\n    accuracy                           0.41       524\n   macro avg       0.17      0.16      0.15       524\nweighted avg       0.35      0.41      0.36       524\n'
Model 2 XGboost Report '              precision    recall  f1-score   support\n\n           1       0.50      0.04      0.07        25\n           2       0.00      0.00      0.00         2\n         

In [8]:
print("Accuracy for model 1: %.2f" % (accuracy_score(y_test, pred1) * 100))
print("Accuracy for model 2: %.2f" % (accuracy_score(y_test, pred2) * 100))

Accuracy for model 1: 41.03
Accuracy for model 2: 41.03


#0기준 이진분류, xgboost로

In [36]:
X_train, X_test, Y_train, Y_test=x_array[:9000], x_array[9000:], y_array[:9000], y_array[9000:]
shuffle_index=np.random.permutation(9000)
X_train, Y_train=X_train[shuffle_index], Y_train[shuffle_index]

Y_train_0=(Y_train==0)#0은 true, 0이 아니면 false
Y_test_0=(Y_test==0)



In [40]:
model1 = xgb.XGBClassifier()
model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.25, subsample=0.5)

train_model1 = model1.fit(X_train, Y_train_0)
train_model2 = model2.fit(X_train, Y_train_0)


In [41]:
pred1 = train_model1.predict(X_test)
pred2 = train_model2.predict(X_test)

print('Model 1 XGboost Report %r' % (classification_report(Y_test_0, pred1)))
print('Model 2 XGboost Report %r' % (classification_report(Y_test_0, pred2)))
print("Accuracy for model 1: %.2f" % (accuracy_score(Y_test_0, pred1) * 100))
print("Accuracy for model 2: %.2f" % (accuracy_score(Y_test_0, pred2) * 100))

Model 1 XGboost Report '              precision    recall  f1-score   support\n\n       False       0.75      0.41      0.53       194\n        True       0.89      0.97      0.93       930\n\n    accuracy                           0.87      1124\n   macro avg       0.82      0.69      0.73      1124\nweighted avg       0.86      0.87      0.86      1124\n'
Model 2 XGboost Report '              precision    recall  f1-score   support\n\n       False       0.63      0.41      0.50       194\n        True       0.88      0.95      0.92       930\n\n    accuracy                           0.86      1124\n   macro avg       0.76      0.68      0.71      1124\nweighted avg       0.84      0.86      0.84      1124\n'
Accuracy for model 1: 87.46
Accuracy for model 2: 85.68


8200/10124*100=80.99~81
, accuracy is higher than 81...괜찮지 않을까...라는 생각을...
learning rate=0.2--> 87%까지 나옴

#0제외한 1924데이터 분류

In [61]:
y_1=y[(y['MRC_ID_DI']==1)]
y_2=y[(y['MRC_ID_DI']==2)]
y_3=y[(y['MRC_ID_DI']==3)]
y_4=y[(y['MRC_ID_DI']==4)]
y_5=y[(y['MRC_ID_DI']==5)]
y_6=y[(y['MRC_ID_DI']==6)]
y_7=y[(y['MRC_ID_DI']==7)]
y_8=y[(y['MRC_ID_DI']==8)]
y_9=y[(y['MRC_ID_DI']==9)]
y_10=y[(y['MRC_ID_DI']==10)]

x_1=x.loc[y_1.index.tolist()]
x_2=x.loc[y_2.index.tolist()]
x_3=x.loc[y_3.index.tolist()]
x_4=x.loc[y_4.index.tolist()]
x_5=x.loc[y_5.index.tolist()]
x_6=x.loc[y_6.index.tolist()]
x_7=x.loc[y_7.index.tolist()]
x_8=x.loc[y_8.index.tolist()]
x_9=x.loc[y_9.index.tolist()]
x_10=x.loc[y_10.index.tolist()]

In [66]:
y_nozero.value_counts()

MRC_ID_DI
7            661
6            554
8            337
5            143
1             85
9             59
3             32
10            28
2             14
4             11
dtype: int64

5,6,7,8

In [83]:
x_5678=x.loc[y_5.index.tolist()+y_6.index.tolist()+y_7.index.tolist()+y_8.index.tolist()]
y_5678=y.loc[x_5678.index.tolist()]
print(x_5678.shape)
print(y_5678.shape)
x_5678=x_5678.to_numpy()
y_5678=y_5678.to_numpy()

(1695, 198)
(1695, 1)


In [84]:
x_5678train, x_5678test, y_5678train, y_5678test=x_5678[:1250], x_5678[1250:], y_5678[:1250], y_5678[1250:]
shuffle_index=np.random.permutation(1250)
x_5678train, y_5678train=x_5678train[shuffle_index], y_5678train[shuffle_index]

In [85]:
train_model1 = model1.fit(x_5678train, y_5678train)
train_model2 = model2.fit(x_5678train, y_5678train)

In [86]:
pred1 = train_model1.predict(x_5678test)
pred2 = train_model2.predict(x_5678test)

print('Model 1 XGboost Report %r' % (classification_report(y_5678test, pred1)))
print('Model 2 XGboost Report %r' % (classification_report(y_5678test, pred2)))
print("Accuracy for model 1: %.2f" % (accuracy_score(y_5678test, pred1) * 100))
print("Accuracy for model 2: %.2f" % (accuracy_score(y_5678test, pred2) * 100))

Model 1 XGboost Report '              precision    recall  f1-score   support\n\n           5       0.00      0.00      0.00         0\n           6       0.00      0.00      0.00         0\n           7       0.45      0.71      0.55       108\n           8       0.00      0.00      0.00       337\n\n    accuracy                           0.17       445\n   macro avg       0.11      0.18      0.14       445\nweighted avg       0.11      0.17      0.13       445\n'
Model 2 XGboost Report '              precision    recall  f1-score   support\n\n           5       0.00      0.00      0.00         0\n           6       0.00      0.00      0.00         0\n           7       0.42      0.72      0.53       108\n           8       0.00      0.00      0.00       337\n\n    accuracy                           0.18       445\n   macro avg       0.11      0.18      0.13       445\nweighted avg       0.10      0.18      0.13       445\n'
Accuracy for model 1: 17.30
Accuracy for model 2: 17.53


이건 아니지이이이잉

#678

In [87]:
x_678=x.loc[y_6.index.tolist()+y_7.index.tolist()+y_8.index.tolist()]
y_678=y.loc[x_678.index.tolist()]
print(x_678.shape)
print(y_678.shape)
x_678=x_678.to_numpy()
y_678=y_678.to_numpy()

(1552, 198)
(1552, 1)


In [93]:
x_678train, x_678test, y_678train, y_678test=x_678[:1230], x_678[1230:], y_678[:1230], y_678[1230:]
#shuffle_index=np.random.permutation(1230)
#x_678train, y_678train=x_678train[shuffle_index], y_678train[shuffle_index]

In [94]:
train_model1 = model1.fit(x_678train, y_678train)
train_model2 = model2.fit(x_678train, y_678train)

In [95]:
pred1 = train_model1.predict(x_678test)
pred2 = train_model2.predict(x_678test)

print('Model 1 XGboost Report %r' % (classification_report(y_678test, pred1)))
print('Model 2 XGboost Report %r' % (classification_report(y_678test, pred2)))
print("Accuracy for model 1: %.2f" % (accuracy_score(y_678test, pred1) * 100))
print("Accuracy for model 2: %.2f" % (accuracy_score(y_678test, pred2) * 100))

Model 1 XGboost Report '              precision    recall  f1-score   support\n\n           6       0.00      0.00      0.00       0.0\n           7       0.00      0.00      0.00       0.0\n           8       0.00      0.00      0.00     322.0\n\n    accuracy                           0.00     322.0\n   macro avg       0.00      0.00      0.00     322.0\nweighted avg       0.00      0.00      0.00     322.0\n'
Model 2 XGboost Report '              precision    recall  f1-score   support\n\n           6       0.00      0.00      0.00       0.0\n           7       0.00      0.00      0.00       0.0\n           8       0.00      0.00      0.00     322.0\n\n    accuracy                           0.00     322.0\n   macro avg       0.00      0.00      0.00     322.0\nweighted avg       0.00      0.00      0.00     322.0\n'
Accuracy for model 1: 0.00
Accuracy for model 2: 0.00


응...내가 병신인걸로...